# Spark ETL Aufgaben
1. Daten von Avro laden und nach Delta rausschreiben


## Wichtige Hinweise
1. Führe alle Anweisungen in der vorgegebenen Reihenfolge aus. Die einzelnen Programmierzellen bauen aufeinander auf.
2. **Beende unbedingt am Ende die Spark-Anwendung mit dem untersten Befehl "spark.stop()" , wenn du aufhörst an den Daten zu arbeiten, damit die Resourcen (cpu und memory) wieder freigegeben werden**
3. Du kannst jederzeit das Notebook wieder hochfahren, wenn du Schritt 1 & 2 (Laden der Imports & Jupyter Spark und seine Konfigurationen hochfahren) ausführen.
4. Mit **"Strg" + "Enter"** führst du einzelne Zellen direkt aus.
5. In der oberen Leiste kannst du über **"Insert"** weitere Zellen hinzufügen, um weitere Test-Funktionen zu schreiben. 

## 1. Module Laden
Hier werden alle benötigten Libraries für dieses Lab heruntergeladen.

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as f

from delta import *


import datetime
from datetime import datetime
import json


# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

## 2.  Spark starten
Hier wird die App jupyter-spark konfiguriert und hochgefahren, welche unsere weiteren Schritte ausführt.

In [ ]:
appName="jupyter-etl"

conf = SparkConf()

# CLUSTER MANAGER
################################################################################
# set Kubernetes Master as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type).
conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")

# CONFIGURE KUBERNETES
################################################################################
# set the namespace that will be used for running the driver and executor pods.
conf.set("spark.kubernetes.namespace","frontend")
# set the docker image from which the Worker pods are created
conf.set("spark.kubernetes.container.image", "thinkportgmbh/workshops:spark-3.3.2")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

# set service account to be used
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
# authentication for service account(required to create worker pods):
conf.set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")


# CONFIGURE SPARK
################################################################################
conf.set("spark.sql.session.timeZone", "Europe/Berlin")
# set driver host. In this case the ingres service for the spark driver
# find name of the driver service with 'kubectl get services' or in the helm chart configuration
conf.set("spark.driver.host", "jupyter-spark-driver.frontend.svc.cluster.local")
# set the port, If this port is busy, spark-shell tries to bind to another port.
conf.set("spark.driver.port", "29413")
# add the postgres driver jars into session
conf.set("spark.jars", "/opt/spark/jars/spark-avro_2.12-3.3.2.jar")
conf.set("spark.executor.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.2.jar")

# CONFIGURE S3 CONNECTOR
conf.set("spark.hadoop.fs.s3a.endpoint", "minio.minio.svc.cluster.local:9000")
conf.set("spark.hadoop.fs.s3a.access.key", "trainadm")
conf.set("spark.hadoop.fs.s3a.secret.key", "train@thinkport")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

#conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions, org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# CONFIGURE WORKER (Customize based on workload)
################################################################################
# set number of worker pods
conf.set("spark.executor.instances", "3")
# set memory of each worker pod
conf.set("spark.executor.memory", "1G")
# set cpu of each worker pod
conf.set("spark.executor.cores", "1")
# Number of possible tasks = cores * executores

## Deltalake
# conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")

# SPARK SESSION
################################################################################
# and last, create the spark session and pass it the config object

spark = SparkSession\
    .builder\
    .config(conf=conf) \
    .config('spark.sql.session.timeZone', 'Europe/Berlin') \
    .appName(appName)\
    .getOrCreate()

# also get the spark context
sc=spark.sparkContext
# change the log level to warning, to see less output
sc.setLogLevel('ERROR')

# get the configuration object to check all the configurations the session was startet with
for entry in sc.getConf().getAll():
        if entry[0] in ["spark.app.name","spark.kubernetes.namespace","spark.executor.memory","spark.executor.cores","spark.driver.host","spark.master"]:
            print(entry[0],"=",entry[1])
            
spark

## 3. ETL: Einlesen - Transformieren - Schreiben

### 3.1 Einlesen der Avro Daten (Extract)
Lade die Daten aus dem Bucket `s3a://twitter/avro` in einen DataFrame

In [ ]:
df_avro=(
    #<< HIER CODE EINFÜGEN >>
    )

# Count und Inhalt anzeigen
print("Anzahl aller Tweets: ",df_avro.count())
df_avro.show(5)

### 3.2 Transformieren des Dataframes (Transform)

1. Filter nur auf die Zeilen, die das Hashtag "BigData" enthalten, mit Hilfe der passenden **Array Function** (https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#array-functions)
2. Benenne folgende Spalten mit kürzeren Namen um, damti später weniger getippt werden muss.
   - `user_name` -> `user`
   - `user_location` -> `country`
   - `user_follower_count` -> `follower`
   - `user_friends_count` -> `friends`
   - `retweet_count` -> `retweets´
3. Füge eine neue Spalte `hastag_count` hinzu in der die Anzahl der Hashtags steht
2. Entferne die Spalte `tweet_message` aus dem Resultset, da diese als langer String viel Speicher benötigt aber für die weiteren Analyse nicht benötigt wird


Dokumentation: https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html

In [ ]:
df_transformed = (df_avro
                  # filter die Zeilen mit Hashtag BigData raus
                  #<< HIER CODE EINFÜGEN >>
                  # benenne die Spalten mit kürzeren Namen um
                  #<< HIER CODE EINFÜGEN >>
                  # füge eine neue Spalte mit der Anzahl der Hasthags hinzu
                  #<< HIER CODE EINFÜGEN >>
                  # falls nicht schon entfernt, entferne die Spalte `tweet_message`
                  #<< HIER CODE EINFÜGEN >>
              )


# Dataframe anzeigen
print("Anzahl aller Tweets: ",df_avro.count())
print("Anzahl nach Transformation: ",df_transformed.count())
df_transformed.show(5)

### 3.3 Rausschreiben der Daten in das Deltaformat (Load)
Schreibe die Daten im Delta Format in das Bucke S3-Bucket `s3a://twitter/delta`   
verwende herbei folgende Spezifikationen:
- partitioniere die Daten nach der Spalte `language` 
- setze den Mode auf `append` (nicht `overwrite`)
- setzt die Option `overwriteSchema`auf `true`
- füge einen Kommentar `load from Spark` in den Metadaten hinzu 

In [ ]:
write=(df_transformed
        .write
        
        # partitioniere nach Spalte `language`
        
        # schreibe mit mode=append
        
        # Füge Optionen und Kommentar dazu
        
        
        .save("WOHIN?")
    )

## 4. Ergebnis überprüfen
Überprüfe mit einer Methode deiner Wahl ob die Daten erfolgreich auf s3 angekommen sind und ob sie Partitioniert wurden

# 6. Ausschalten der Spark-App
**Bitte schließe am Ende die Spark-App wieder mit dem folgenden Befehl `spark.stop()`, wenn du fertig mit der Bearbeitung der Aufgaben bist.** 

In [ ]:
spark.stop()